<a href="https://colab.research.google.com/github/Papa-Panda/random_thoughts/blob/main/CNN_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

# Define the transform to normalize the input data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-10 dataset and apply the transform
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Create a DataLoader to load the dataset in batches
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)


Files already downloaded and verified


In [4]:
# Define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the CNN
net = Net()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the CNN
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')


[1,  1000] loss: 2.251
[1,  2000] loss: 1.925
[1,  3000] loss: 1.808
[1,  4000] loss: 1.662
[1,  5000] loss: 1.556
[1,  6000] loss: 1.480
[1,  7000] loss: 1.439
[1,  8000] loss: 1.399
[1,  9000] loss: 1.376
[1, 10000] loss: 1.322
[1, 11000] loss: 1.348
[1, 12000] loss: 1.283
[2,  1000] loss: 1.210
[2,  2000] loss: 1.166
[2,  3000] loss: 1.115
[2,  4000] loss: 1.107
[2,  5000] loss: 1.081
[2,  6000] loss: 1.069
[2,  7000] loss: 1.031
[2,  8000] loss: 1.030
[2,  9000] loss: 1.010
[2, 10000] loss: 0.981
[2, 11000] loss: 0.982
[2, 12000] loss: 0.967
[3,  1000] loss: 0.869
[3,  2000] loss: 0.853
[3,  3000] loss: 0.884
[3,  4000] loss: 0.863
[3,  5000] loss: 0.866
[3,  6000] loss: 0.818
[3,  7000] loss: 0.864
[3,  8000] loss: 0.859
[3,  9000] loss: 0.825
[3, 10000] loss: 0.848
[3, 11000] loss: 0.818
[3, 12000] loss: 0.792
[4,  1000] loss: 0.705
[4,  2000] loss: 0.704
[4,  3000] loss: 0.712
[4,  4000] loss: 0.708
[4,  5000] loss: 0.712
[4,  6000] loss: 0.749
[4,  7000] loss: 0.698
[4,  8000] 